In [18]:
!pip3 install opencv-python
import cv2
import numpy as np
import math



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
def _crop_to_signature(gray, thresh_val=220):
    """
    Crop the image to the bounding box of the signature ink.
    Assumes light background, dark strokes.
    """
    # Binary image: background white(255), signature black(0) after inversion
    _, th = cv2.threshold(gray, thresh_val, 255, cv2.THRESH_BINARY_INV)
    
    # Find all non-zero points (signature area)
    coords = cv2.findNonZero(th)
    if coords is None:
        # No ink detected, return original
        return gray
    
    x, y, w, h = cv2.boundingRect(coords)
    cropped = gray[y:y+h, x:x+w]
    return cropped




In [20]:
def _deskew(gray):
    """
    Deskew the signature using image moments.
    Works best when background is clean and signature is main object.
    """
    # Threshold to isolate signature
    _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    coords = np.column_stack(np.where(th > 0))
    if coords.size == 0:
        return gray
    
    # Fit a min-area rectangle around the ink
    rect = cv2.minAreaRect(coords.astype(np.float32))
    angle = rect[-1]
    
    # Correct OpenCV's angle convention
    if angle < -45:
        angle = 90 + angle
    
    # Small angles not worth rotating
    if abs(angle) < 1:
        return gray
    
    (h, w) = gray.shape
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(gray, M, (w, h),
                             flags=cv2.INTER_CUBIC,
                             borderMode=cv2.BORDER_REPLICATE)
    return rotated


In [ ]:
def preprocess_signature(
    img_input,
    size=224,
    mean=0.5,
    std=0.5,
    deskew=True,
    crop=True,
    as_tensor=False
):
    """
    Best-practice preprocessing for signature images.
    
    Args:
        img_input: str path OR numpy array (BGR or grayscale).
        size: final output size (size x size).
        mean, std: normalization params for model input (after 0-1 scaling).
        deskew: whether to attempt deskewing the signature.
        crop: whether to crop to the tight bounding box around the signature.
        as_tensor: if True, output shape is (1, H, W) for CNN input.
        
    Returns:
        img_norm: float32 array in shape (H, W) or (1, H, W),
                  normalized and ready for model.
    """
    # 1. Load image as grayscale
    if isinstance(img_input, str):
        gray = cv2.imread(img_input, cv2.IMREAD_GRAYSCALE)
        if gray is None:
            raise ValueError(f"Could not read image from path: {img_input}")
    else:
        img = img_input
        # If BGR, convert to grayscale
        if len(img.shape) == 3 and img.shape[2] == 3:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        elif len(img.shape) == 3 and img.shape[2] == 1:
            gray = img[:, :, 0]
        else:
            gray = img.copy()

    # 2. Slight blur to reduce sensor/paper noise (very light)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    # 3. Optional crop to signature content
    if crop:
        gray = _crop_to_signature(gray)

    # 4. Optional deskew
    if deskew:
        gray = _deskew(gray)

    # 5. Resize with aspect ratio preserved + pad to square
    h, w = gray.shape
    if h == 0 or w == 0:
        raise ValueError("Empty image encountered after cropping/deskewing.")

    scale = size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(gray, (new_w, new_h), interpolation=cv2.INTER_AREA)

    h2, w2 = resized.shape
    pad_top = (size - h2) // 2
    pad_bottom = size - h2 - pad_top
    pad_left = (size - w2) // 2
    pad_right = size - w2 - pad_left

    padded = cv2.copyMakeBorder(
        resized,
        pad_top, pad_bottom, pad_left, pad_right,
        borderType=cv2.BORDER_CONSTANT,
        value=255  # white background
    )

    # 6. Convert to float and scale to [0, 1]
    img_float = padded.astype(np.float32) / 255.0

    # 7. Normalize with mean/std (like standard CNN input)
    # Result is roughly in range [-1, 1] if mean=0.5, std=0.5
    img_norm = (img_float - mean) / std

    # 8. Add channel dimension if using with CNNs (C, H, W)
    if as_tensor:
        img_norm = np.expand_dims(img_norm, axis=0)

    return img_norm


In [23]:
processed = preprocess_signature("sig1.jpg")
cv2.imwrite("processed_sig1.png", processed[0]*255)

True